In [3]:
#test the model class on JSWT simulations 
import numpy as np
import pandas as pd
#import splat
#import popsims
from astropy.coordinates import SkyCoord, Galactic
#from popsims.galaxy import Pointing, volume_calc, create_pop
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
#sns.set_style("darkgrid", {"axes.facecolor": ".95"})

#plt.style.use('fivethirtyeight')
#plt.style.use('dark_background')

from  matplotlib.colors import Normalize
import astropy.units as u
import popsims
import matplotlib
from tqdm import tqdm
from tqdm import tqdm
import matplotlib as mpl 
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] =20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['legend.fontsize'] = 20
mpl.rcParams['figure.titlesize'] = 20

from popsims.simulator import Population#,simulate_population
from popsims.galaxy import Disk, Halo
from popsims.relations import polynomial_relation
#from popsims.backgrounds import simulate_lsst_background_stars, read_isochrones

In [ ]:
popsims.simulator.POLYNOMIALS['absmags_spt']['subdwarfs']

In [ ]:
#bhjk

In [ ]:
#LOAD DP0 data
dp0_data=pd.read_csv('/Volumes/lacie/popsimsdata/cone_search.csv')

In [ ]:
#randoom 1,000 poinitings
dp0_data_rand= dp0_data.sample(n=1000)
footprint=SkyCoord(ra=dp0_data_rand.ra*u.deg, dec=dp0_data_rand.dec*u.deg)

In [ ]:
fig, ax=plt.subplots()
ax.scatter(dp0_data.ra, dp0_data.dec, s=.1, alpha=0.02)
ax.scatter(footprint.ra, footprint.dec, s=10, c='k')

In [ ]:
#simulate a disk population
mag_keys=['LSST_R', 'LSST_I', 'LSST_Z' ]

In [ ]:
mag_keys

In [ ]:
bckg=pd.concat([simulate_lsst_background_stars(1e2, imf='kroupa', mass_range=(0.1, 1), \
                                               met_range=(0.0, 0.5)) for i in tqdm(range(0, 10))])

In [ ]:
#bckg

In [ ]:
disk= Disk()
samples_tot= simulate_population(disk, footprint, 0.1, 15000, poptype='dwarfs', mag_keys=mag_keys, \
                             galtype='thin_disk', nsample=5e4, get_from='spt')

In [ ]:
mag_keys

In [ ]:
#thick disk
disk= Disk(H=300, L=3600)
samples_thick_tot= simulate_population(disk, footprint, 0.1, 15000, poptype='subdwarfs', \
                                   galtype='thick_disk' , mag_keys=mag_keys, evolmodel='burrows1997',\
                                   age_range=[8, 13], nsample=5e4, get_from='spt')

In [ ]:
#### thick disk
halo=Halo()
samples_halo_tot= simulate_population(halo, footprint, 0.1, 15000, poptype='subdwarfs', galtype='halo', \
                                   age_range=[10, 14], mag_keys=mag_keys,  evolmodel='burrows1997',
                                  nsample=5e4, get_from='spt')

In [ ]:
# u : 23.9,  26.1
#g : 25.0,  27.4
#r : 24.7, 27.5
#i : 24.0 , 26.8
#z : 23.3,  26.1
#y : 22.1, 24.9
mag_limits_single=dict(zip(mag_keys, [25.0, 24.7, 24.0, 23.3, 22.1]))
mag_limits_10yr=dict(zip(mag_keys, [27.4, 27.5, 26.8, 26.1, 24.9]))
mag_limits= mag_limits_single

In [ ]:
for k in [ 'LSST_R', 'LSST_I', 'LSST_Z']:
    print (k, mag_limits[k])
    samples= samples_tot[np.logical_and(samples_tot[k]<mag_limits[k], samples_tot[k] >1) ].reset_index(drop=True)
    samples_thick= samples_thick_tot[np.logical_and(samples_thick_tot[k]<mag_limits[k], samples_thick_tot[k]>1)].reset_index(drop=True)
    samples_halo= samples_halo_tot[np.logical_and(samples_halo_tot[k]<mag_limits[k], samples_halo_tot[k]>1) ].reset_index(drop=True)
    bckg= bckg[np.logical_and(bckg[k]<mag_limits[k], bckg[k] >10)].reset_index(drop=True)

In [ ]:
for s in [samples, samples_thick, samples_halo, bckg]:
    #s['g-r']=s.abs_LSST_G-s.abs_LSST_R
    s['i-z']=s.abs_LSST_I-s.abs_LSST_Z
    s['r-i']=s.abs_LSST_R-s.abs_LSST_I
    #s['z-y']=s.abs_LSST_Z-s.abs_LSST_Y
    #s['g-i']=s.abs_LSST_G-s.abs_LSST_I
    s['r-z']=s.abs_LSST_R-s.abs_LSST_Z
    

In [ ]:
#bckg= bckg[np.logical_and(bckg['i-z']>0, bckg['r-i']>0)]

In [ ]:
fig, ax=plt.subplots()


ax.scatter(dp0_data.mag_i-dp0_data.mag_z, dp0_data.mag_r-dp0_data.mag_i, color='k', label='DP0 Data', s=.1)
ax.scatter(bckg['i-z'], bckg['r-i'], color='grey', label='Simulated bkgd', s=1)

ax.scatter(samples['i-z'], samples['r-i'], s=1, marker=',', \
           edgecolor='#0074D9',  facecolor='none', alpha=0.2, label='Thin Disk')
ax.scatter(samples_thick['i-z'], samples_thick['r-i'], alpha=0.2, s=10,\
           edgecolor='#FF851B', facecolor='none', marker='*', label='Thick Disk')
#ax.scatter(samples_halo['i-z'], samples_halo['r-i'],  s=10,\
#           edgecolor='#B10DC9',  facecolor='none', alpha=0.2, marker='*', label='Halo')

ax.axvline(0.3, linestyle='--', color='k')
ax.axhline(0.2, linestyle='--', color='k')

ax.minorticks_on()
ax.legend(fontsize=16, markerscale=3)

ax.set(xlim=[-2, 4.], ylim=[-2, 4.0], xlabel='i-z', ylabel='r-i')

plt.savefig('../figures/lsst_dp0_simulations_ri_iz.jpeg')

In [ ]:
fig, ax=plt.subplots()

ax.scatter(dp0_data.mag_r-dp0_data.mag_z, dp0_data.mag_r-dp0_data.mag_z, color='k', label='LSST DP0', s=.1)
#ax.scatter(bckg['r-i'], bckg['g-r'], color='grey', label='Simulated bkgd', s=10)

ax.scatter(samples['r-z'][samples.spt<20], samples['r-z'][samples.spt<20],  s=10, marker=',', alpha=0.5, color='#0074D9',  label='Thin Disk')
#ax.scatter(samples_thick['r-z'], samples_thick['g-r'], s=10, marker='*', color='#FF851B',  label='Thick Disk')
#ax.scatter(samples_halo['r-z'], samples_halo['g-r'],  s=10, marker='*', color='#B10DC9', label='Halo')

ax.axvline(0.3, linestyle='--', color='k')
ax.axhline(0.2, linestyle='--', color='k')

ax.minorticks_on()
legend=ax.legend(fontsize=16, markerscale=6)

ax.set( xlabel='r-z', ylabel='g-r', xlim=[-5, 5], ylim=[-5, 5.])

plt.savefig('../figures/lsst_dp0_simulations_rz_gr.jpeg')

In [ ]:
#adsfvd

In [ ]:
fig, ax=plt.subplots()

#ax.scatter(dp0_data.mag_r-dp0_data.mag_z, dp0_data.mag_g-dp0_data.mag_r, color='k', label='DP0 Data', s=.1)
ax.scatter(bckg.abs_LSST_R-bckg.abs_LSST_Z, bckg.abs_LSST_R, color='k', label='Simulated bkgd', s=1)

ax.scatter(samples['r-z'], samples.abs_LSST_R,  s=1, marker=',', alpha=0.5, color='#0074D9',  label='Dwarfs')
ax.scatter(samples_thick['r-z'], samples_thick.abs_LSST_R, s=1, marker='*', color='#FF851B',  label='Subdwarfs')
#ax.scatter(samples_halo['r-z'], samples_halo.abs_LSST_R, s=10, marker='*', color='#B10DC9', label='Halo')

#ax.axvline(0.3, linestyle='--', color='k')
#ax.axhline(0.2, linestyle='--', color='k')

ax.minorticks_on()
legend=ax.legend(fontsize=16, markerscale=3)

ax.set( xlabel='r-z', ylabel=r'M$_r$', xlim=[0.5, 6.], ylim=[10, 21])
ax.invert_yaxis()


In [ ]:
bckg.columns

In [ ]:
fig, ax=plt.subplots()

#ax.scatter(dp0_data.mag_r-dp0_data.mag_z, dp0_data.mag_g-dp0_data.mag_r, color='k', label='DP0 Data', s=.1)
#ax.scatter(bckg['r-z'], bckg['g-r'], color='grey', label='Simulated bkgd', s=1)

ax.scatter(samples['r-z'], samples.spt,  s=1, marker=',', alpha=0.5, color='#0074D9',  label='Dwarfs')
ax.scatter(samples_thick['r-z'], samples_thick.spt ,s=10, marker='*', color='#FF851B',  label='Subdwarfs')
#ax.scatter(samples_halo['r-z'], samples_halo.abs_LSST_R, s=10, marker='*', color='#B10DC9', label='Halo')

#ax.axvline(0.3, linestyle='--', color='k')
#ax.axhline(0.2, linestyle='--', color='k')

ax.minorticks_on()
legend=ax.legend(fontsize=16, markerscale=3)

ax.set( xlabel='r-z', ylabel=r'Spectral Type', xlim=[0.5, 6.], ylim=[10, 40])


In [ ]:
def compute_vols_and_numbers(df, sptgrid, footprint, maglimits):
    counts={}
    vols={}
    dists={}
    
    disk= Disk()
    tdisk=  Disk(H=900, L=3600)
    halo= Halo()
    
    
    for spt in tqdm(sptgrid):
        
        dmins=[]
        dmaxs=[]
        
        dmins_sd=[]
        dmaxs_sd=[]
        
        for k in maglimits.keys():
            mag_cut= maglimits[k]
            absmag= np.poly1d(popsims.simulator.POLYNOMIALS['absmags_spt']['dwarfs'][k]['fit'])(spt)
            absmag_sd= np.poly1d(popsims.simulator.POLYNOMIALS['absmags_spt']['subdwarfs'][k]['fit'])(spt)
        
            mag_cut= maglimits[k]
            
            dmin=10.**(-(absmag-14)/5. + 1.)
            dmax=10.**(-(absmag-mag_cut)/5. + 1.)
            
            dmin_sd=10.**(-(absmag_sd-14)/5. + 1.)
            dmax_sd=10.**(-(absmag_sd-mag_cut)/5. + 1.)
            
            
            
            dmins.append(dmin)
            dmaxs.append(dmax)
            
            dmins_sd.append(dmin)
            dmaxs_sd.append(dmax)
            
        dmin=np.nanmedian(dmins)
        dmax=np.nanmedian(dmaxs)
        
        dmin_sd=np.nanmedian(dmins_sd)
        dmax_sd=np.nanmedian(dmaxs_sd)
        
        #print (spt, dmin, dmax)
        
        scale=[df.scale.mean(), df.scale_unc.mean(), df.scale_times_model.mean()]
        
        sn= len(df.query('population == "thin disk"'))
        snt= len(df.query('population == "thick disk"'))
        snh= len(df.query('population == "halo"'))
      
        sn_c= len(df.query('population == "thin disk" and spt >= {} and spt < {}'.format(spt, spt+0.9)))
        snt_c= len(df.query('population == "thick disk" and spt >= {} and spt < {}'.format(spt, spt+0.9)))
        snh_c= len(df.query('population == "halo" and spt >= {} and spt < {}'.format(spt, spt+0.9)))
        
        
        volumes={'thin': 0., 'thick':0., 'halo': 0.}
        
        cnts={'thin':  sn_c*np.divide(scale[-1], sn),
             'thick': snt_c*np.divide(scale[-1], snt),\
             'halo':  snh_c*np.divide(scale[-1], snh)}

        for s in  footprint:
            l=s.galactic.l.radian
            b=s.galactic.b.radian
            volumes['thin'] += disk.volume(l, b, dmin, dmax)/len(footprint)
            volumes['thick'] += tdisk.volume(l, b, dmin_sd, dmax_sd)/len(footprint)
            volumes['halo'] += halo.volume(l, b, dmin_sd, dmax_sd)/len(footprint)
            
        vols.update({spt: volumes})
        counts.update({spt: cnts})
        dists.update({spt: dmax})
        
        
    return pd.DataFrame.from_records(vols).T.replace(np.inf, np.nan),\
    pd.DataFrame.from_records(counts).T.replace(np.inf, np.nan),\
    dists


In [ ]:
samples_tot['population']='thin disk'
samples_thick_tot['population']='thick disk'
samples_halo_tot['population']='halo'

In [ ]:
mag_limits=mag_limits_single
for k in ['LSST_R', 'LSST_I', 'LSST_Z']:
    samples= samples_tot[np.logical_and(samples_tot[k]<mag_limits[k], samples_tot[k] >1) ].reset_index(drop=True)
    samples_thick= samples_thick_tot[np.logical_and(samples_thick_tot[k]<mag_limits[k], samples_thick_tot[k]>1)].reset_index(drop=True)
    samples_halo= samples_halo_tot[np.logical_and(samples_halo_tot[k]<mag_limits[k], samples_halo_tot[k]>1) ].reset_index(drop=True)
samples['population']='thin disk'
samples_thick['population']='thick disk'
samples_halo['population']='halo'
df= pd.concat([samples, samples_thick, samples_halo])
sptgrid=np.arange(15, 40)

#Add astrometric cut
#addd astrometric cut
astro_cut=(df.mu_alpha_cosdec**2+df.mu_delta**2)**0.5 >10
ns=compute_vols_and_numbers(df, sptgrid, footprint, mag_limits)
ns_astro=compute_vols_and_numbers(df[astro_cut], sptgrid, footprint, mag_limits)

mag_limits=mag_limits_10yr
for k in ['LSST_R', 'LSST_I', 'LSST_Z']:
    samples= samples_tot[np.logical_and(samples_tot[k]<mag_limits[k], samples_tot[k] >1) ].reset_index(drop=True)
    samples_thick= samples_thick_tot[np.logical_and(samples_thick_tot[k]<mag_limits[k], samples_thick_tot[k]>1)].reset_index(drop=True)
    samples_halo= samples_halo_tot[np.logical_and(samples_halo_tot[k]<mag_limits[k], samples_halo_tot[k]>1) ].reset_index(drop=True)
samples['population']='thin disk'
samples_thick['population']='thick disk'
samples_halo['population']='halo'
df= pd.concat([samples, samples_thick, samples_halo])
sptgrid=np.arange(15, 40)
ns_10yr=compute_vols_and_numbers(df, sptgrid, footprint, mag_limits)
astro_cut=(df.mu_alpha_cosdec**2+df.mu_delta**2)**0.5 >10
ns_10yr_astro=compute_vols_and_numbers(df[astro_cut], sptgrid, footprint, mag_limits)


In [4]:
AREA=(18_000*(u.degree**2).to(u.radian**2)*(u.radian**2))

In [ ]:
def tick_function(locs):
    return [x*18_1000 for x in locs]

In [ ]:
fig, ax=plt.subplots(figsize=(17.5, 7), ncols=2, sharey=True)

ax[0].step(ns[0].index, ns[0].thin*ns[1].thin*AREA,\
        color='#0074D9',  linestyle='-', label='Thin Disk', linewidth=3)
ax[0].step(ns[0].index, 0.12*ns[0].thick*ns[1].thick*AREA,\
         color='#0074D9',  linestyle='dashed', label='Thick Disk',  linewidth=3)
ax[0].step(ns[0].index, (1/400)*ns[0].thick*ns[1].halo*AREA,\
           color='#0074D9', linestyle='dotted', label='Halo',  linewidth=3)
ax[0].set(yscale='log', ylabel=r'Surface Density (deg$^{-2}$)', xlabel='Spectral Type', title='5-Sigma Depth')
ax[0].legend(fontsize=12)

ax[1].step(ns_10yr[0].index, ns_10yr[0].thin*ns_10yr[1].thin*AREA,\
        color='#0074D9',  linestyle='-', label='Thin Disk', linewidth=3)
ax[1].step(ns_10yr[0].index, 0.12*ns_10yr[0].thick*ns_10yr[1].thick*AREA,\
         color='#0074D9',  linestyle='dashed', label='Thick Disk', linewidth=3)
ax[1].step(ns_10yr[0].index, (1/400)*ns_10yr[0].thick*ns_10yr[1].halo*AREA,\
           color='#0074D9', linestyle='dotted', label='Halo', linewidth=3)
ax[1].set(yscale='log', ylabel=r'Surface Density (deg$^{-2}$)', xlabel='Spectral Type', title='10 yr Depth')


#ax[0].step(ns_astro[0].index, ns_astro[0].thin*ns_astro[1].thin*AREA,\
#        color='#FF851B',  linestyle='-', label='Thin Disk + Astrometric Detection', linewidth=3)
#ax[0].step(ns[0].index, 0.12*ns_astro[0].thick*ns_astro[1].thick*AREA,\
#         color='#FF851B',  linestyle='dashed', label='Thick Disk + Astrometric Detection',  linewidth=3)
#ax[0].step(ns[0].index, (1/400)*ns_astro[0].thick*ns_astro[1].halo*AREA,\
#           color='#FF851B', linestyle='dotted', label='Halo + Astrometric Detection',  linewidth=3)

ax[0].set(yscale='log', ylabel=r'Surface Density (deg$^{-2}$)', xlabel='Spectral Type', title='Single Exposure')
#ax[0].legend(fontsize=12)

#ax[1].step(ns_10yr[0].index, ns_10yr_astro[0].thin*ns_10yr_astro[1].thin*AREA,\
#        color='#FF851B',  linestyle='-', label='Thin Disk + Astrometric Detection', linewidth=3)
#ax[1].step(ns_10yr[0].index, 0.12*ns_10yr_astro[0].thick*ns_10yr_astro[1].thick*AREA,\
#         color='#FF851B',  linestyle='dashed', label='Thick Disk + Astrometric Detection', linewidth=3)
#ax[1].step(ns_10yr[0].index, (1/400)*ns_10yr_astro[0].thick*ns_10yr_astro[1].halo*AREA,\
#           color='#FF851B', linestyle='dotted', label='Halo + Astrometric Detection', linewidth=3)
#ax[1].set(yscale='log', ylabel=r'Surface Density (deg$^{-2}$)', xlabel='Spectral Type', title='10 yr Depth')

for a in ax :
    ax2 = a.twinx()
    #ax2.set_yticks(new_tick_locations)
    ax2.set_yticklabels(["{:.0e}".format(x) for x in new_tick_locations*18_0000])
    #ax2.tick_params(colors='#0074D9', which='both') 
    ax2.set_ylabel('Number Counts')
    #ax2.grid()


ax[1].legend(fontsize=12)
ax[1].yaxis.set_tick_params(labelbottom=True)
plt.tight_layout()
ax[0].set(xticks=[ 15,20, 25, 30, 35, 39], xticklabels=['M5', 'L0', 'L5', 'T0', 'T5', 'T9'])
ax[1].set(xticks=[15,20, 25, 30, 35, 39], xticklabels=['M5', 'L0', 'L5', 'T0', 'T5', 'T9'])
ax[0].minorticks_on()
ax[-1].minorticks_on()
plt.savefig('../figures/number_counts_lsst.jpeg')

In [ ]:
mask=np.logical_and(ns_10yr[0].index>=20, ns_10yr[0].index <30)
print ('L dwarfs with detectable motion')
np.nansum((ns_10yr_astro[0].thin*ns_10yr_astro[1].thin*AREA)[mask])

In [ ]:
mask=np.logical_and(ns_10yr[0].index>=30, ns_10yr[0].index <=40)
print ('L dwarfs with detectable motion')
np.nansum((ns_10yr_astro[0].thin*ns_10yr_astro[1].thin*AREA)[mask])

In [ ]:
bckg.age.min()

In [ ]:
import astropy.units as u
((32*u.arcmin)**2).to(u.deg**2)